In [ ]:
import networkx as nx
import pickle as pkl
import numpy as np
import argparse
import shutil
import sys
import os

In [ ]:
def create_dynamic_graph(edges, window_size): 
    '''
    returns a dynamic graph
    inputs: 
        edges (|E| x 3 matrix) : each row represents a temporal edge (u, v, t)
        widnow_size (int) : the time range of each graph in the dynamic graph
    
    outputs: 
        graphs (list of static graphs)
    '''
    
    def process_edges(edges): 
        '''first sort edges by time, then remap edge times to integer increments ie. [0, total_timesteps]'''
        # sort edges by time
        edges = edges[np.argsort(edges[:, 2]), :]
        
        # remap edge times to integer increments
        edges_remapped = np.zeros(edges.shape)
        t_original = edges[0, 2]
        t_remapped = 0
        for t in range(edges.shape[0]): 
            if edges[t, 2] > t_original: 
                t_original = edges[t, 2]
                t_remapped += 1

            edges_remapped[t, :] = np.array([edges[t, 0], edges[t, 1], t_remapped])

        return edges_remapped
    
    edges = process_edges(edges)
    graphs = []
    edge_times = edges[:, 2]
    total_timesteps = np.unique(edge_times).shape[0] // window_size # floor divide ignores the remainder timesteps
    
    # iterate through temporal edges and create total_timesteps static graphs
    for t in range(total_timesteps): 
        # create a mask that selects edges and nodes within the range [t * window_size, (t+1) * window_size]
        t_start = t * window_size
        t_end = (t + 1) * window_size
        mask_start = (edge_times >= t_start).astype(int)
        mask_end = (edge_times < t_end).astype(int)
        mask = (mask_start + mask_end) == 2
        edges_t = edges[mask, :2]
        
        # create a static graph using all the edges and nodes within the range [t * window_size, (t+1) * window_size]
        g = nx.Graph()
        g.add_edges_from(edges_t)
        graphs.append(g)
    
    return graphs

In [ ]:
data = 0 # integers 0-6 map to dataset choices
dataset_params = {0: {'path': 'CollegeMsg.txt', 'window_size': 2000}, # https://snap.stanford.edu/data/CollegeMsg.html
                  1: {'path': 'email-Eu-core-temporal.txt', 'window_size': 5000}, # https://snap.stanford.edu/data/email-Eu-core.html
                  2: {'path': 'sx-mathoverflow.txt', 'window_size': 12000}, # https://snap.stanford.edu/data/sx-mathoverflow.html
                  3: {'path': 'soc-sign-bitcoinalpha.csv', 'window_size': 100}, # https://snap.stanford.edu/data/soc-sign-bitcoin-alpha.html
                  4: {'path': 'soc-sign-bitcoinotc.csv', 'window_size': 1000}, # https://snap.stanford.edu/data/soc-sign-bitcoin-otc.html
                  5: {'path': 'soc-redditHyperlinks-body.tsv', 'window_size': 10000},
                  6: {'path': 'soc-redditHyperlinks-title.tsv', 'window_size': 10000}}

# fill in the data path and download datasets to path
# path = ...

if data in [0, 1, 2]:
    edges = np.loadtxt(f'{path}/{dataset_params[real]["path"]}')
elif data in [3, 4]:
    edges = np.genfromtxt(f'{path}/{dataset_params[real]["path"]}', delimiter=',')[:, [0, 1, 3]]
elif data in [5, 6]:
    edges = pd.read_csv(f'{path}/{dataset_params[real]["path"]}', delimiter='\t')

graphs = create_dynamic_graph(edges, window_size=window_size)